In [1]:
import json
import os
import xml.etree.ElementTree as ET

# Import warnings module and ignore warnings in output below
import warnings
warnings.filterwarnings("ignore")

# Import and initialize MAAP class
from maap.maap import MAAP
maap = MAAP(maap_host="sister-api.imgspec.org")

Set up workflow config

In [6]:
input_config =  [
    {
      "step_key": "l1_preprocess",
      "algorithm_id": "sister-l1_preprocess_ubuntu",
      "version": "master",
      "queue": "sister-job_worker-32gb",
      "params": {
        "l1_granule": "https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20200917091806_20200917091810_0001.zip",
        "resolution": 30,
        "landsat": "s3://sister-ops-workspace/prisma/landsat_reference/PRS_20200917091806_20200917091810_0001_landsat.tar.gz",
        "publish_to_cmr": False,
        "cmr_metadata": {},
        "identifier": "cwl-workflow-test-l1_preprocess"
      }
    },
    {
      "step_key": "l2_reflectance",
      "algorithm_id": "isofit_ubuntu",
      "queue": "sister-job_worker-32gb",
      "version": "cwl-dev",
      "input_filter": {
        "l1_granule": "*.tar.gz"
      },
      "params": {
        "cmr_metadata": {},
        "l1_granule": None,
        "n_cores": 32,
        "publish_to_cmr": False,
        "radiance_factors_file": "https://raw.githubusercontent.com/EnSpec/sister/master/data/prisma/radiance_factors/PRISMA_20200721104249_20200721104253_0001_radiance_factors.txt",
        "segmentation_size": 50,
        "snow_and_liquids_reflectance_spectra": "https://ecosis.org/api/package/emit-manually-adjusted-snow-and-liquids-reflectance-spectra/export",
        "surface_reflectance_spectra": "https://ecosis.org/api/package/emit-manually-adjusted-surface-reflectance-spectra/export",
        "vegetation_reflectance_spectra": "https://ecosis.org/api/package/emit-manually-adjusted-vegetation-reflectance-spectra/export",
        "water_reflectance_spectra": "https://ecosis.org/api/package/emit-manually-adjusted-water-reflectance-spectra/export"
      },
      "identifier": "cwl-workflow-test-l2_reflectance"
    },
    {
      "step_key": "l2_resample",
      "algorithm_id": "sister-resample_ubuntu",
      "version": "sister-dev",
      "queue": "sister-job_worker-32gb",
      "input_filter": {
        "l2_granule": "*rfl*.tar.gz"
      },
      "params": {
        "l2_granule": None,
        "publish_to_cmr": False,
        "cmr_metadata": {},
        "identifier": "cwl-workflow-test-l2_resample"
      }
    },
    {
      "step_key": "l2_reflectance_correction",
      "algorithm_id": "sister-reflect_correct_ubuntu",
      "version": "main",
      "queue": "sister-job_worker-32gb",
      "input_filter": {
        "l1_granule": "*l1p.tar.gz",
        "l2_granule": "*rfl.tar.gz"
      },
      "params": {
        "l1_granule": None,
        "l2_granule": None,
        "publish_to_cmr": False,
        "cmr_metadata": {},
        "identifier": "cwl-workflow-test-l2_reflectance_correction"
      }
    }
  ]


Submit the workflow job

In [7]:
job = maap.submitJob(
    algo_id="run_sister_workflow_ubuntu",
    version="dev",
    queue="sister-job_worker-8gb",
    identifier="sister-cwl-workflow-test",
    username="anonymous",
    workflow_config=json.dumps(input_config)
),
job


({'job_id': '045f1743-2a5d-499a-a8a4-d3f958c0c127', 'status': 'success', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': []},)